In [51]:
# Standard library imports
import os
import sys

# Third-party imports
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer, AerSimulator

# Adjust the system path to include the project root
# This allows for importing from other directories within the project structure.
project_root = os.path.abspath("..")  # Assumes this script is one level from the root
sys.path.append(project_root)


In [57]:
def initialize(n_bits):
    """ 
    Randomly generates the qubits and the bases used by Alice and Bob.

    This function initializes the simulation by generating random bits for Alice and random bases for both Alice and Bob. 
    Bases are represented as integers, where '0' denotes the rectilinear basis and '1' denotes the diagonal (Hadamard) basis.

    """
    Alice_bits = np.random.randint(2,size=n_bits)
    Alice_bases = np.random.randint(2,size=n_bits)
    Bob_bases = np.random.randint(2,size=n_bits)
    
    return Alice_bits, Alice_bases, Bob_bases

def Alice_qc(Alice_bits, Alice_bases):
    """ 
    Prepare quantum circuits for Alice based on her bits and chosen bases.

    This function creates a list of quantum circuits for each bit in Alice's sequence. Each circuit is prepared according to the value of the bit and the basis:
    - If the bit is 1, an X gate (Pauli-X) is applied to flip the qubit from 0 to 1.
    - If the basis is 1 (Hadamard basis), a Hadamard (H) gate is applied to put the qubit into superposition.
    Otherwise, the qubit is prepared in the standard basis without any additional gates for a bit of 0.

    """
    circuits = []
    for bit, Alice_basis in zip(Alice_bits, Alice_bases):
        qc = QuantumCircuit(1,1)
        if bit == 1:
            qc.x(0)
        if Alice_basis == 1:
            qc.h(0)
        circuits.append(qc)
    return circuits


def Bob_qc(circuits, Bob_bases):
    """ 
    Prepare and measure quantum circuits based on Bob's chosen measurement bases.

    This function processes a list of quantum circuits by applying the necessary basis transformation and measurement operations according to Bob's bases. For each circuit:
    - If Bob's basis is 1 (diagonal basis), a Hadamard gate (H) is applied to transform the basis from the computational (Z) to the diagonal (X) basis.
    - Every qubit is then measured in its respective basis, with the measurement result being stored in a classical bit.

    The function modifies the circuits in-place and does not return any value.

    """
    if len(circuits) != len(Bob_bases):
        raise RuntimeError("circuits and Bob_bases should have the same length")
    
    for qc, Bob_basis in zip(circuits, Bob_bases):
        # Bob prepares the measurement and basis
        if Bob_basis == 1:
            qc.h(0)
            
        qc.measure(0, 0) # Bob measures the qubit and stores the result in clbit  
          
    return None
        
def run_circuit(qc):
    """ 
    Simulate a quantum circuit and return the measurement outcome.

    This function takes a quantum circuit, transpiles it for optimal performance on a simulator, and then runs the simulation to measure the state of the qubit. It returns the outcome of this measurement. The simulation is run with only one measurement shot to directly observe the state of the qubit.
    """
    
    # Simulate the circuit
    simulator = AerSimulator()

    transpiled_qc = transpile(qc) # Transpile optimizes circuits, adapts it to real world devices
    result = simulator.run(transpiled_qc, shots=1).result()
    counts = result.get_counts()
    measured_bit = int(list(counts.keys())[0])  # Get the measurement outcome
    
    return measured_bit

def sift_key(Alice_bits, Alice_bases, Bob_bases, measured_bits):
    ''' 
    Generate the secret key from matched bases between Alice and Bob.

    This function compares the bases used by Alice and Bob to determine where they match. If the bases match, it means that the corresponding bits (from Alice's original bits and Bob's measured bits) are used to form part of the secret key. This sifting process is crucial in quantum key distribution (QKD) to ensure that both parties have identical keys formed from bits that were reliably transmitted and measured under the same basis.

    '''
    Alices_key = []
    Bobs_key = [] 
    matching_indices = []
    for i in range(len(Alice_bits)):
        if Alice_bases[i] == Bob_bases[i]:
            Alices_key.append(Alice_bits[i])
            Bobs_key.append(measured_bits[i])
            matching_indices.append(i)
            
    return Alices_key, Bobs_key, matching_indices
    

def bb84_qkd():
    """ 
    Simulate the BB84 quantum key distribution protocol to generate a shared secret key.

    This function simulates the BB84 protocol by:
    1. Initializing random bits and bases for Alice and random bases for Bob.
    2. Creating quantum circuits based on Alice's bits and bases, which are then sent to Bob.
    3. Bob measures the received qubits using his bases.
    4. Both Alice and Bob use the sifting process to extract a shared secret key from bits measured under matching bases.

    The simulation prints each step, showing the bit sent by Alice, the bases used by both, and the bit measured by Bob. This allows for tracing the protocol's progress and understanding how mismatches and errors affect the final key.

    """
    n_bits = 32 # Number of qubits Alice will send
    Alice_bits, Alice_bases, Bob_bases = initialize(n_bits)
    
    circuits = Alice_qc(Alice_bits, Alice_bases)
    Bob_qc(circuits, Bob_bases)
    
    
    measured_bits = []
    for bit, Alice_basis, Bob_basis, qc in zip(Alice_bits, Alice_bases, Bob_bases, circuits):
        measured_bit = run_circuit(qc)
        measured_bits.append(measured_bit)
        print(f"The bit sent by Alice was: {bit}")
        print(f"The basis used by Alice was: {Alice_basis}")
        print(f"The basis used by Bob was: {Bob_basis}")
        print(f"The measured bit was: {measured_bit} \n")
        
    Alice_key, Bob_key, matching_indices = sift_key(Alice_bits, Alice_bases, Bob_bases, measured_bits)

    return Alice_key, Bob_key, matching_indices

Alice_key, Bob_key, matching_indices = bb84_qkd()
print(list(map(int, Alice_key)))
print(Bob_key)


The bit sent by Alice was: 0
The basis used by Alice was: 1
The basis used by Bob was: 0
The measured bit was: 1 

The bit sent by Alice was: 0
The basis used by Alice was: 1
The basis used by Bob was: 0
The measured bit was: 1 

The bit sent by Alice was: 1
The basis used by Alice was: 0
The basis used by Bob was: 0
The measured bit was: 1 

The bit sent by Alice was: 0
The basis used by Alice was: 0
The basis used by Bob was: 0
The measured bit was: 0 

The bit sent by Alice was: 1
The basis used by Alice was: 0
The basis used by Bob was: 1
The measured bit was: 0 

The bit sent by Alice was: 0
The basis used by Alice was: 0
The basis used by Bob was: 0
The measured bit was: 0 

The bit sent by Alice was: 1
The basis used by Alice was: 1
The basis used by Bob was: 1
The measured bit was: 1 

The bit sent by Alice was: 0
The basis used by Alice was: 0
The basis used by Bob was: 0
The measured bit was: 0 

The bit sent by Alice was: 1
The basis used by Alice was: 0
The basis used by Bo